# Ciała skończone

In [47]:
def ExtendedGCD(a,b):
    r,r1=a,b
    s,s1=1,0 #s*a+t*b == a
    t,t1=0,1 #s1*a+t1*b == b
    while not(r1==0):
        q,r2=r//r1,r % r1
        r,s,t,r1,s1,t1=r1,s1,t1,r2,-q*s1+s,-q*t1+t
    d=r
    return d,s,t #s*a+t*b=d, d=GCD(a,b)

def MultInverse(a,n):
    d,inv,_=ExtendedGCD(a,n)
    if d==1:
        if n==1:
            return 1 #for compatibility
        return inv%n
    else:
        raise NameError('Numbers '+str(a)+' and '+str(n)+' are not coprime.')
        
def power_fast(x, n):
    if n < 0:
        x = x**(-1)
        n = -n
    if n == 0:
        return x*(x**(-1))
    y = Fin(1,x.p)
    while n > 1:
        if n % 2 == 0:
            x = x * x
            n = n // 2;
        else:
            y = x * y;
            x = x * x;
            n = (n - 1) // 2
    return x * y
        
class Fin():
    def __init__(self,a,modulus):
        self.x=a%modulus
        self.p=modulus #to musi być liczba pierwsza
        self.parent=FiniteField(self.p)
    
    def __add__(self,b):
        if type(b)==Fin:
            return Fin(((self.x)+b.x)%self.p,self.p)
        if type(b)==int:
            return Fin(((self.x)+b)%self.p,self.p)
        if type(b)==polynomial:
            return b+self
    def __sub__(self,b):
        if type(b)==Fin:
            return Fin(((self.x)-b.x)%self.p,self.p)
        if type(b)==int:
            return Fin(((self.x)-b)%self.p,self.p)
        if type(b)==polynomial:
            return -b+self
    def __mul__(self,b):
        if type(b)==Fin:
            return Fin(((self.x)*b.x)%self.p,self.p)
        if type(b)==int:
            return Fin(((self.x)*b)%self.p,self.p)
        if type(b)==polynomial:
            return b*self

    def __pow__(self,k):
        if self.x==0:
            return Fin(0,self.p)
        if k==1:
            return self
        if k==-1:
            return Fin(MultInverse(self.x,self.p),self.p)
        if k<=-2:
            inv=Fin(MultInverse(self.x,self.p),self.p)
            return power_fast(inv,k)
        if k>=2:
            return power_fast(self,k)
    def __str__(self):
        return str(self.x)
    def __repr__(self):
        return str(self.x)
    def __print__(self):
        print(str(self.x))
    
    def __eq__(self,b):
        if (self.p==b.p) and (self.x==b.x):
            return True
        else:
            return False
        
    def __div__(self,b):
        if not(b.x==0):
            return self*(b**(-1))
        else:
            raise Exception('Dzielenie przez 0.')
    def __floordiv__(self,b):
        if not(b.x==0):
            return self*(b**(-1))
        else:
            raise Exception('Dzielenie przez 0.')

    def __neg__(self):
        return Fin(-self.x,self.p)
    
class FiniteField():
    def __init__(self,p):
        self.p=p
        
    def __str__(self):
        return "Ciało skończone {}-elementowe".format(self.p)
    def __repr__(self):
        return "Ciało skończone {}-elementowe".format(self.p)
    
    def __call__(self,a):
        return Fin(a,self.p)
    def __eq__(self,F):
        if type(self)==type(F):
            if self.p==F.p:
                return True
        return False

In [55]:
F=FiniteField(11)
a=F(5)
type(a)
a**-1
F(1)//a

9

# Pierścienie wielomianów

In [49]:
def ExtendedGCDPol(a,b):
    r,r1=a,b
    s,s1=polynomial([a.one],a.type),polynomial([a.zero],a.type) #s*a+t*b == a
    t,t1=polynomial([a.zero],a.type),polynomial([a.one],a.type) #s1*a+t1*b == b
    while not(r1==a.zero):
        q,r2=r//r1,r % r1
        r,s,t,r1,s1,t1=r1,s1,t1,r2,s-s1*q,t-t1*q
    d=r
    return d,s,t #s*a+t*b=d, d=GCD(a,b)

def power_fast_pol(x, n):
    y = polynomial([x.one],x.type)
    if n==0:
        return y
    while n > 1:
        if n % 2 == 0:
            x = x * x
            n = n // 2;
        else:
            y = x * y;
            x = x * x;
            n = (n - 1) // 2
    return x * y

class polynomial():
    def __init__(self,pollist,cootype=int,var='x'):
        
        if all(isinstance(n, type(cootype(0))) for n in pollist):
            #wytnij koncowe zera
            self.zero=cootype(0)
            self.one=cootype(1)
            while len(pollist)>1 and pollist[-1]==self.zero:
                pollist.pop()
            self.coo=pollist #lista współczynników
            self.deg=len(pollist)-1
            self.type=cootype
            self.ctype=type(cootype(0))
        else:
            self.zero=cootype(0)
            self.one=cootype(1)
            self.coo=[cootype(1)]
            self.deg=0
            self.type=cootype
            self.ctype=type(cootype(0))
        self.var=var
        
    def __str__(self):
        i=0
        if not(self.coo[0]==self.zero):
            polprint=str(self.coo[0])
            zerostarted=False
            iszero=False
        else:
            polprint=''
            zerostarted=True
            iszero=True
        for el in self.coo[1:]:
            i+=1
            if not(el==self.zero):
                iszero=False
                if not zerostarted:
                    if el == self.one:
                        polprint+=' + {}^{}'.format(self.var,str(i))
                    else:
                        polprint+=' + {}*{}^{}'.format(str(el),self.var,str(i))
                else:
                    if el == self.one:
                        polprint+='{}^{}'.format(self.var,str(i))
                    else:
                        polprint+='{}*{}^{}'.format(str(el),self.var,str(i))
                    zerostarted=False
        if iszero:
            return '0'
        else:
            return polprint
        
    def __repr__(self):
        return self.__str__()
        
    def __add__(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        
        li1=self.coo
        li2=pol.coo
        if not (self.type==pol.type):
            raise TypeError("Wielomiany nad różnymi pierścieniami.")
        m1=len(li1)
        m2=len(li2)
        li3=[]
        if m2>m1:
            li1,li2=li2,li1
            m1,m2=m2,m1
        for i in range(0,m2):
            li3.append(li1[i]+li2[i])
        for i in range(m2,m1):
            li3.append(li1[i])
        return polynomial(li3,self.type,self.var)
    
    def __sub__(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        li1=self.coo
        li2=pol.coo
        if not (self.type==pol.type):
            raise TypeError("Wielomiany nad różnymi pierścieniami.")
        m1=len(li1)
        m2=len(li2)
        li3=[]
        if m1<m2:
            for i in range(0,m1):
                li3.append(li1[i]-li2[i])
            for i in range(m1,m2):
                li3.append(-li2[i])
        else:
            for i in range(0,m2):
                li3.append(li1[i]-li2[i])
            for i in range(m2,m1):
                li3.append(li1[i])
            
        return polynomial(li3,self.type,self.var)
    
    def __mul__(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        li1=self.coo
        li2=pol.coo
        if not (self.type==pol.type):
            raise TypeError("Wielomiany nad różnymi pierścieniami.")
        if (li1==[self.zero]) or (li2==[self.zero]):
            return polynomial([self.zero],self.type,self.var)
        l1=self.deg+1
        l2=pol.deg+1
        li3=[self.zero]*(l1+l2-1)
        for i in range(0,l1):
            for j in range(0,l2):
                li3[i+j]+=li1[i]*li2[j]
        return polynomial(li3,self.type,self.var)
    
    def __eq__(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        if self.coo==pol.coo:
            return True
        else:
            return False        
    def __neg__(self):
        li=[]
        for i in range(0,self.deg+1):
            li.append(self.coo[i]*(-self.one))
        return polynomial(li,self.type,self.var)
    
    def __pow__(self,k):
        if self.deg==0 and self.coo[0]==self.zero:
            return polynomial([self.zero],self.type,self.var)
        if k==1:
            return self
        if k<=-1:
            raise Exception('Nie ma odwrotności.')
        if k>=2:
            return power_fast_pol(self,k)
        
    def quorem(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        if self.deg <pol.deg:
            return polynomial([self.zero],self.type,self.var),self
        t=self.one//(pol.coo[pol.deg]) #musi istnieć odwrotność
        r=[]
        q=[self.zero]*(self.deg-pol.deg+1)
        for i in range(0,self.deg+1):
            r.append(self.coo[i])
        for i in range(self.deg-pol.deg,-1,-1):
            q[i]=t*r[i+pol.deg]
            for j in range(0,pol.deg+1):
                r[i+j]-=q[i]*pol.coo[j]
        return polynomial(q,self.type,self.var), polynomial(r,self.type,self.var)
    def __mod__(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        _,r=self.quorem(pol)
        return r
    def __floordiv__(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        q,_=self.quorem(pol)
        return q
    
    def ExtendedGCD(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        return ExtendedGCDPol(self,pol)
        
    def GCD(self,pol):
        if type(pol)==int:
            pol=polynomial([self.type(pol)],self.type,self.var)
        if type(pol)==self.ctype:
            pol=polynomial([pol],self.type,self.var)
        d,_,_=ExtendedGCDPol(self,pol)
        return d
    
class PolynomialRing():
    def __init__(self,ring,var='x'):
        self.R=ring
        self.x=polynomial([ring(0),ring(1)],ring,var)
        self.var=var
    
    def __str__(self):
        return "Pierścień wielomianów nad {} zmiennej {}".format(self.R,self.var)
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self,li):
        if type(li)==list:
            return polynomial([self.R(x) for x in li],self.R,self.var)
        if type(li)==int:
            return polynomial([self.R(li)],self.R,self.var)
    def __eq__(self,ring):
        if type(self)==type(ring):
            if self.R==ring.R:
                return True
        return False

In [68]:
F=FiniteField(2)
R=PolynomialRing(F,'x')
x=R.x

In [78]:
f=x**5+x+1
g=x**6+1
print(f*g)
#print(pol1+pol1)
pol1-pol2
pol1-R([1,2,3,4])
pol1**100
(pol1**5)%pol2
(pol1**7).quorem(pol2)

1 + x^1 + x^5 + x^6 + x^7 + x^11


(1 + x^1 + x^2 + x^3 + x^4 + x^5 + x^6 + x^7, 0)

In [59]:
F=FiniteField(2)
RF=PolynomialRing(F,'y')
y=RF.x
pol1=RF([1,0,0,0,1])
pol2=RF([1,2,1])
print(pol1)
print(pol2)
pol2.ExtendedGCD(pol1)

1 + y^4
1 + y^2


(1 + y^2, 1, 0)

In [534]:
RZ=PolynomialRing(int,'z')
z=RZ.x
pol1=RZ([1,2,3,1])
pol2=RZ([3,5,7,1])
pol1.quorem(pol2)
pol1%pol2
#uwaga - wybrana implementacja GCD musi być poprawiona dla int - to nie jest ciało

-2 + -3*z^1 + -4*z^2

# Pierścienie ilorazowe

In [247]:
def power_fast_pol_mod(x, n,modulus):
    y = polynomial([x.one],x.type)
    if n==0:
        return y
    while n > 1:
        if n % 2 == 0:
            x = (x * x)%(modulus)
            n = n // 2;
        else:
            y = (x * y)%(modulus);
            x = (x * x)%(modulus);
            n = (n - 1) // 2
    return x * y

class QuoRingEl():
    def __init__(self,pol,modulus,var='x'):
        self.pol=pol%modulus
        self.modulus=modulus
        self.var=var
        
    def __add__(self,p):
        if type(p)==type(self):
            return QuoRingEl(self.pol+p.pol,self.modulus,self.var)
        return QuoRingEl(self.pol+p,self.modulus,self.var)
    
    def __mul__(self,p):
        if type(p)==type(self):
            return QuoRingEl(self.pol*p.pol,self.modulus,self.var)
        return QuoRingEl(self.pol*p,self.modulus,self.var)
    
    def __sub__(self,p):
        if type(p)==type(self):
            return QuoRingEl(self.pol-p.pol,self.modulus,self.var)
        return QuoRingEl(self.pol-p,self.modulus,self.var)
    
    def __truediv__(self,p):
        if type(p)==type(self):
            d,s,t=p.pol.ExtendedGCD(self.modulus)
            #a/b == a*s (mod n)
            #s*b+t*n==d
            #(s/d)*b+(t/d)*n==1
            if d.deg>0 or d==d.zero:
                raise NameError('Nie ma odwrotności.')
            return QuoRingEl(self.pol*s//d.coo[0],self.modulus,self.var)
        d,t,s=self.modulus.ExtendedGCD(p)
        if d.deg>0 or d==d.zero:
                raise NameError('Nie ma odwrotności.')
        return QuoRingEl(self.pol*s//d.coo[0],self.modulus,self.var)
        
    
    def __eq__(self,p):
        if type(self)==type(p):
            if self.pol==p.pol:
                return True
        return False
    def __neg__(self):
        return QuoRingEl(-self.pol,modulus,self.var)
    
    def __pow__(self,k):
        if k<0:
            pp=QuoRingEl(polynomial([self.pol.one],self.pol.type),self.modulus,self.var)/self
            return QuoRingEl(power_fast_pol_mod(pp.pol,-k,self.modulus),self.modulus,self.var)
        return QuoRingEl(power_fast_pol_mod(self.pol,k,self.modulus),self.modulus,self.var)
    
    def __str__(self):
        i=0
        if not(self.pol.coo[0]==self.pol.zero):
            polprint=str(self.pol.coo[0])
            zerostarted=False
            iszero=False
        else:
            polprint=''
            zerostarted=True
            iszero=True
        for el in self.pol.coo[1:]:
            i+=1
            if not(el==self.pol.zero):
                iszero=False
                if not zerostarted:
                    if el == self.pol.one:
                        polprint+=' + {}^{}'.format(self.var,str(i))
                    else:
                        polprint+=' + {}*{}^{}'.format(str(el),self.var,str(i))
                else:
                    if el == self.pol.one:
                        polprint+='{}^{}'.format(self.var,str(i))
                    else:
                        polprint+='{}*{}^{}'.format(str(el),self.var,str(i))
                    zerostarted=False
        if iszero:
            if SHOWMOD==True:
                return '0 (mod {})'.format(self.modulus)
            else:
                return '0'
        else:
            if SHOWMOD==True:
                return polprint+' (mod {})'.format(self.modulus)
            else:
                return polprint
        
    def __repr__(self):
        return self.__str__()
    
class QuotientRing():
    def __init__(self,polR,modulus,var='x'):
        self.reprR=polR
        self.modulus=modulus
        self.x=QuoRingEl(polR.x,modulus,var)
        self.var=var
    
    def __str__(self):
        return "Pierścień ilorazowy {} modulo {}".format(self.reprR,self.modulus)
    
    def __repr__(self):
        return self.__str__()
    
    def __call__(self,pol):
        if type(pol)==int:
            return QuoRingEl(polynomial([self.reprR.R(pol)],self.reprR.R),self.modulus,self.var)
        return QuoRingEl(pol,self.modulus,self.var)
    def __eq__(self,ring):
        if type(self)==type(ring):
            if (self.reprR==ring.reprR) and (self.modulus==ring.modulus):
                return True
        return False
        

# Przykłady

In [248]:
F=FiniteField(2)
R=PolynomialRing(F)
x=R.x
pol=x**2+x+1

In [249]:
pol.ExtendedGCD(1)

(1, 0, 1)

In [250]:
SHOWMOD=False
F4=QuotientRing(R,pol,'a')
a=F4.x
F4el=[F4(0),F4(1),a,a+1]

In [251]:
a**(-1)

1 + a^1

In [552]:
#Tabliczka dodawania
[["{}+{}={}".format(a,b,a+b) for b in F4el] for a in F4el]

[['0+0=0', '0+1=1', '0+a^1=a^1', '0+1 + a^1=1 + a^1'],
 ['1+0=1', '1+1=0', '1+a^1=1 + a^1', '1+1 + a^1=a^1'],
 ['a^1+0=a^1', 'a^1+1=1 + a^1', 'a^1+a^1=0', 'a^1+1 + a^1=1'],
 ['1 + a^1+0=1 + a^1', '1 + a^1+1=a^1', '1 + a^1+a^1=1', '1 + a^1+1 + a^1=0']]

In [553]:
#Tabliczka mnożenia
[["({})*({})={}".format(a,b,a*b) for b in F4el] for a in F4el]

[['(0)*(0)=0', '(0)*(1)=0', '(0)*(a^1)=0', '(0)*(1 + a^1)=0'],
 ['(1)*(0)=0', '(1)*(1)=1', '(1)*(a^1)=a^1', '(1)*(1 + a^1)=1 + a^1'],
 ['(a^1)*(0)=0', '(a^1)*(1)=a^1', '(a^1)*(a^1)=1 + a^1', '(a^1)*(1 + a^1)=1'],
 ['(1 + a^1)*(0)=0',
  '(1 + a^1)*(1)=1 + a^1',
  '(1 + a^1)*(a^1)=1',
  '(1 + a^1)*(1 + a^1)=a^1']]

In [555]:
R2=PolynomialRing(F4,var='T')

In [556]:
T=R2.x

In [557]:
(T+a)*(T+(a+1)) #faktoryzacja wielomianu 1+T+T^2 nad ciałem 4-elementowym

1 + T^1 + T^2

Nierozkładalne wielomiany nad ciałem $2$-elementowym

In [452]:
F=FiniteField(2)
R=PolynomialRing(F)
x=R.x

In [235]:
nst1=[x+1]
nst2=[x**2+x+1]
nst30=[x**3+F(a)*x**2+F(b)*x+1 for a in range(0,2) for b in range(0,2)]

In [238]:
[[p%(x**2+x+1),p%(x+1),p] for p in nst3]

[[0, 0, 1 + x^3],
 [x^1, 1, 1 + x^1 + x^3],
 [1 + x^1, 1, 1 + x^2 + x^3],
 [1, 0, 1 + x^1 + x^2 + x^3]]

In [240]:
nst30=[x**3+x+1,x**3+x**2+1]

In [252]:
F8=QuotientRing(R,x**3+x+1,'b')
b=F8.x

In [253]:
[b**k for k in range(0,7)] #hence b is a generator of the multiplicative group

[1, b^1, b^2, 1 + b^1, b^1 + b^2, 1 + b^1 + b^2, 1 + b^2]

In [346]:
F=FiniteField(2)
R=PolynomialRing(F)
x=R.x
def IrredPol2(n,x):
    if n==1:
        return [x+1]
    if n>1:
        li=[]
        for a in range(0,2**(n-1)):
            licoo=list(map(int,list(bin(a))[2:]))
            licoo.reverse()
            inpol=R(licoo)*x
            pol=x**n+inpol+1
            #print("a=",a)
            #print("pol=",pol)
            isred=False
            for k in range(1,n//2+1):
                #print("k=",k)
                irrk=IrredPol2(k,x)
                #print("irrk=",irrk)
                for el in irrk:
                    #print("el=",el)
                    #print("pol%el=",pol%el)
                    if pol%el==F(0):
                        #print("isred")
                        isred=True
                        break
                if isred:
                    #print("Przerwij k petle")
                    break
            if not isred:
                li.append(pol)
        return li
                    
                

In [501]:
IrredPol2(8,x)

[1 + x^1 + x^3 + x^4 + x^8,
 1 + x^2 + x^3 + x^4 + x^8,
 1 + x^1 + x^3 + x^5 + x^8,
 1 + x^2 + x^3 + x^5 + x^8,
 1 + x^3 + x^4 + x^5 + x^8,
 1 + x^1 + x^2 + x^3 + x^4 + x^5 + x^8,
 1 + x^2 + x^3 + x^6 + x^8,
 1 + x^1 + x^2 + x^3 + x^4 + x^6 + x^8,
 1 + x^1 + x^5 + x^6 + x^8,
 1 + x^2 + x^5 + x^6 + x^8,
 1 + x^3 + x^5 + x^6 + x^8,
 1 + x^4 + x^5 + x^6 + x^8,
 1 + x^1 + x^2 + x^4 + x^5 + x^6 + x^8,
 1 + x^1 + x^3 + x^4 + x^5 + x^6 + x^8,
 1 + x^1 + x^2 + x^7 + x^8,
 1 + x^1 + x^3 + x^7 + x^8,
 1 + x^2 + x^3 + x^7 + x^8,
 1 + x^1 + x^2 + x^3 + x^4 + x^7 + x^8,
 1 + x^1 + x^5 + x^7 + x^8,
 1 + x^3 + x^5 + x^7 + x^8,
 1 + x^4 + x^5 + x^7 + x^8,
 1 + x^2 + x^3 + x^4 + x^5 + x^7 + x^8,
 1 + x^1 + x^6 + x^7 + x^8,
 1 + x^1 + x^2 + x^3 + x^6 + x^7 + x^8,
 1 + x^1 + x^2 + x^4 + x^6 + x^7 + x^8,
 1 + x^2 + x^3 + x^4 + x^6 + x^7 + x^8,
 1 + x^1 + x^2 + x^5 + x^6 + x^7 + x^8,
 1 + x^1 + x^4 + x^5 + x^6 + x^7 + x^8,
 1 + x^2 + x^4 + x^5 + x^6 + x^7 + x^8,
 1 + x^3 + x^4 + x^5 + x^6 + x^7 + x^8]

In [472]:
#Rijnadel vel AES - wybór wielomianu jak poniżej
F256=QuotientRing(R,x**8+x**4+x**3+x+1,'c')
c=F256.x

def HexToF256(n):
    xl=list(map(int,bin(int(n,16))[2:]))
    xl.reverse()
    return F256(R(xl))
def F256ToHex(a):
    al=a.pol.coo.copy()
    al.reverse()
    return hex(int("".join(map(str,al)),2))[2:].upper()

In [473]:
HexToF256('57')*HexToF256('83')

1 + c^6 + c^7

In [436]:
HexToF256('C1')

1 + c^6 + c^7

In [437]:
F256ToHex(HexToF256('57')*HexToF256('83'))

'C1'

In [438]:
HexToF256('09')**(-1)

1 + c^1 + c^2 + c^3 + c^6

In [439]:
F256ToHex(HexToF256('FF'))

'FF'

In [440]:
F256ToHex(HexToF256('57')*HexToF256('13'))

'FE'

In [441]:
F256ToHex(HexToF256('17')*HexToF256('A3'))

'9A'

In [442]:
F256ToHex(HexToF256('F7')**6)

'C3'

In [443]:
F256ToHex(HexToF256('12')**6)

'3B'

In [466]:
F256ToHex(F256(1))

'1'

In [464]:
[F256ToHex(c**k) for k in range(0,15)]

['1',
 '2',
 '4',
 '8',
 '10',
 '20',
 '40',
 '80',
 '1B',
 '36',
 '6C',
 'D8',
 'AB',
 '4D',
 '9A']

In [477]:
F256ToHex(HexToF256('09')**(-1))

'4F'